In [60]:
import sys
import pandas as pd
import numpy as np
import math
from scipy.special import softmax
from scipy.optimize import minimize, LinearConstraint
from scipy.spatial.distance import braycurtis
import multiprocessing
from concurrent.futures import ProcessPoolExecutor, as_completed
from imports import *
from importlib import reload
# # adding Folder_2 to the system path
# sys.path.insert(0, r'C:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\code')
# # sys.path.insert(0, r'C:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\code')

from imports import  *

import social_model_class
reload(social_model_class)


<module 'social_model_class' from 'c:\\Users\\tomer\\Desktop\\BSc\\year3\\sem B\\workshop_microbiome\\Milestone2\\social_model_class.py'>

In [61]:
data_path = r"train_data.csv"
metadata_path = r"train_metadata.csv"

$$\Delta_t  \space windowing$$

In [62]:
model = social_model_class.superModel(data_path, metadata_path)

## cross validation (4 - fold)

In [73]:
np.random.seed(0)
baboons = model.baboons
dict_list = list(model.baboons.items())
np.random.shuffle(dict_list)
groups = [dict(group) for group in np.array_split(dict_list,4)]

data_df = pd.read_csv(data_path, index_col=0)
metadata_df = pd.read_csv(metadata_path, index_col=0)

for i in range(4):
    train_metatdata = metadata_df[~metadata_df["baboon_id"].isin(groups[i].keys())]
    train_data = data_df.loc[train_metatdata.index]
    known_metadata = metadata_df[metadata_df['baboon_id'].isin(groups[i].keys())]
    indicies = []
    for baboon in groups[i].values():
        indicies.extend(baboon.metadata_I.index[:10])

    known_data = data_df.loc[indicies]
    true_data = data_df.loc[known_metadata[~np.isin(known_metadata.index, indicies)].index]


    train_data.to_csv(fr".\cross_val_data\train_data_{i}.csv")
    train_metatdata.to_csv(fr".\cross_val_data\train_metadata_{i}.csv")
    known_data.to_csv(fr".\cross_val_data\known_data_{i}.csv")
    known_metadata.to_csv(fr".\cross_val_data\known_metadata_{i}.csv")
    true_data.to_csv(fr".\cross_val_data\true_data_{i}.csv")
    

In [74]:
i= 0
model = social_model_class.superModel(fr".\cross_val_data\train_data_{i}.csv", fr".\cross_val_data\train_metadata_{i}.csv")
model.alpha_ = np.array([0]*61)
model.add_new_data(fr".\cross_val_data\known_data_{i}.csv", fr".\cross_val_data\known_metadata_{i}.csv")



baboon_id:  Baboon_113
baboon_id:  Baboon_136
baboon_id:  Baboon_201
baboon_id:  Baboon_238
baboon_id:  Baboon_285
baboon_id:  Baboon_343
baboon_id:  Baboon_368
baboon_id:  Baboon_374
baboon_id:  Baboon_378
baboon_id:  Baboon_379
baboon_id:  Baboon_388
baboon_id:  Baboon_4
baboon_id:  Baboon_450
baboon_id:  Baboon_491
baboon_id:  Baboon_494
baboon_id:  Baboon_499
baboon_id:  Baboon_51
baboon_id:  Baboon_518
baboon_id:  Baboon_544
baboon_id:  Baboon_64


In [75]:
groups[i].keys()

dict_keys(['Baboon_518', 'Baboon_379', 'Baboon_388', 'Baboon_64', 'Baboon_491', 'Baboon_494', 'Baboon_285', 'Baboon_368', 'Baboon_51', 'Baboon_374', 'Baboon_450', 'Baboon_238', 'Baboon_113', 'Baboon_499', 'Baboon_544', 'Baboon_4', 'Baboon_136', 'Baboon_343', 'Baboon_201', 'Baboon_378'])

In [77]:
predictions_noniterative = model.predict(groups[i].keys(), iterative=False)

predicting:  Baboon_518
10 56
predicting:  Baboon_379
10 78
predicting:  Baboon_388
10 59
predicting:  Baboon_64
10 96
predicting:  Baboon_491
10 73
predicting:  Baboon_494
10 130
predicting:  Baboon_285
10 87
predicting:  Baboon_368
10 54
predicting:  Baboon_51
10 113
predicting:  Baboon_374
10 79
predicting:  Baboon_450
10 61
predicting:  Baboon_238
10 59
predicting:  Baboon_113
10 131
predicting:  Baboon_499
10 83
predicting:  Baboon_544
10 57
predicting:  Baboon_4
10 122
predicting:  Baboon_136
10 59
predicting:  Baboon_343
10 60
predicting:  Baboon_201
10 81
predicting:  Baboon_378
10 64


c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\social_model_class.py:206: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return predictions


In [78]:
for i in range(4):
    model = social_model_class.superModel(fr".\cross_val_data\train_data_{i}.csv", fr".\cross_val_data\train_metadata_{i}.csv")
    model.alpha_ = np.array([0]*61)
    
    cpus = max(1, min(multiprocessing.cpu_count() - 2, len(model.baboons)))
    with ProcessPoolExecutor(cpus) as executor:
        futures = [executor.submit(baboon.fit, model.alpha_) for baboon in model.baboons.values()]
        for future in as_completed(futures):
            pass
    
    print(f"predicting {i}")
    
    model.add_new_data(fr".\cross_val_data\known_data_{i}.csv", fr".\cross_val_data\known_metadata_{i}.csv")
    predictions_noniterative = model.predict(groups[i].keys(), iterative=False)
    predictions_iterative = model.predict(groups[i].keys(), iterative=True)
    predictions_noniterative.to_csv(fr".\cross_val_data\predictions_noniterative_{i}.csv")
    predictions_iterative.to_csv(fr".\cross_val_data\predictions_iterative_{i}.csv")

predicting 0
baboon_id:  Baboon_113
baboon_id:  Baboon_136
baboon_id:  Baboon_201
baboon_id:  Baboon_238
baboon_id:  Baboon_285
baboon_id:  Baboon_343
baboon_id:  Baboon_368
baboon_id:  Baboon_374
baboon_id:  Baboon_378
baboon_id:  Baboon_379
baboon_id:  Baboon_388
baboon_id:  Baboon_4
baboon_id:  Baboon_450
baboon_id:  Baboon_491
baboon_id:  Baboon_494
baboon_id:  Baboon_499
baboon_id:  Baboon_51
baboon_id:  Baboon_518
baboon_id:  Baboon_544
baboon_id:  Baboon_64
predicting:  Baboon_518
10 56
predicting:  Baboon_379
10 78
predicting:  Baboon_388
10 59
predicting:  Baboon_64
10 96
predicting:  Baboon_491
10 73
predicting:  Baboon_494
10 130
predicting:  Baboon_285
10 87
predicting:  Baboon_368
10 54
predicting:  Baboon_51
10 113
predicting:  Baboon_374
10 79
predicting:  Baboon_450
10 61
predicting:  Baboon_238
10 59
predicting:  Baboon_113
10 131
predicting:  Baboon_499
10 83
predicting:  Baboon_544
10 57
predicting:  Baboon_4
10 122
predicting:  Baboon_136
10 59
predicting:  Baboon_3

c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\social_model_class.py:206: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return predictions


predicting:  Baboon_518
10 56
predicting:  Baboon_379
10 78
predicting:  Baboon_388
10 59
predicting:  Baboon_64
10 96
predicting:  Baboon_491
10 73
predicting:  Baboon_494
10 130
predicting:  Baboon_285
10 87
predicting:  Baboon_368
10 54
predicting:  Baboon_51
10 113
predicting:  Baboon_374
10 79
predicting:  Baboon_450
10 61
predicting:  Baboon_238
10 59
predicting:  Baboon_113
10 131
predicting:  Baboon_499
10 83
predicting:  Baboon_544
10 57
predicting:  Baboon_4
10 122
predicting:  Baboon_136
10 59
predicting:  Baboon_343
10 60
predicting:  Baboon_201
10 81
predicting:  Baboon_378
10 64


c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\social_model_class.py:206: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return predictions


predicting 1
baboon_id:  Baboon_109
baboon_id:  Baboon_153
baboon_id:  Baboon_199
baboon_id:  Baboon_200
baboon_id:  Baboon_249
baboon_id:  Baboon_253
baboon_id:  Baboon_298
baboon_id:  Baboon_349
baboon_id:  Baboon_382
baboon_id:  Baboon_389
baboon_id:  Baboon_395
baboon_id:  Baboon_439
baboon_id:  Baboon_464
baboon_id:  Baboon_479
baboon_id:  Baboon_502
baboon_id:  Baboon_549
baboon_id:  Baboon_595
baboon_id:  Baboon_599
baboon_id:  Baboon_61
predicting:  Baboon_479
10 57
predicting:  Baboon_439
10 51
predicting:  Baboon_464
10 108
predicting:  Baboon_534
9 72
predicting:  Baboon_595
10 59
predicting:  Baboon_298
10 53
predicting:  Baboon_109
10 63
predicting:  Baboon_199
10 56
predicting:  Baboon_200
10 86
predicting:  Baboon_599
10 42
predicting:  Baboon_502
10 56
predicting:  Baboon_153
10 98
predicting:  Baboon_395
10 93
predicting:  Baboon_549
10 67
predicting:  Baboon_382
10 123
predicting:  Baboon_253
10 53
predicting:  Baboon_61
10 112
predicting:  Baboon_349
10 86
predicting

...

c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\social_model_class.py:206: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return predictions


predicting 3
baboon_id:  Baboon_206
baboon_id:  Baboon_209
baboon_id:  Baboon_245
baboon_id:  Baboon_258
baboon_id:  Baboon_259
baboon_id:  Baboon_280
baboon_id:  Baboon_324
baboon_id:  Baboon_333
baboon_id:  Baboon_342
baboon_id:  Baboon_373
baboon_id:  Baboon_410
baboon_id:  Baboon_42
baboon_id:  Baboon_441
baboon_id:  Baboon_458
baboon_id:  Baboon_488
baboon_id:  Baboon_492
baboon_id:  Baboon_503
baboon_id:  Baboon_506
baboon_id:  Baboon_570
baboon_id:  Baboon_78
predicting:  Baboon_333
10 101
predicting:  Baboon_503
10 113
predicting:  Baboon_206
10 45
predicting:  Baboon_259
10 114
predicting:  Baboon_373
10 72
predicting:  Baboon_42
10 55
predicting:  Baboon_492
10 79
predicting:  Baboon_441
10 89
predicting:  Baboon_280
10 55
predicting:  Baboon_78
10 113
predicting:  Baboon_258
10 73
predicting:  Baboon_458
10 88
predicting:  Baboon_410
10 66
predicting:  Baboon_342
10 74
predicting:  Baboon_209
10 70
predicting:  Baboon_570
10 54
predicting:  Baboon_324
10 51
predicting:  Babo

c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\social_model_class.py:206: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return predictions


predicting:  Baboon_333
10 101
predicting:  Baboon_503
10 113
predicting:  Baboon_206
10 45
predicting:  Baboon_259
10 114
predicting:  Baboon_373
10 72
predicting:  Baboon_42
10 55
predicting:  Baboon_492
10 79
predicting:  Baboon_441
10 89
predicting:  Baboon_280
10 55
predicting:  Baboon_78
10 113
predicting:  Baboon_258
10 73
predicting:  Baboon_458
10 88
predicting:  Baboon_410
10 66
predicting:  Baboon_342
10 74
predicting:  Baboon_209
10 70
predicting:  Baboon_570
10 54
predicting:  Baboon_324
10 51
predicting:  Baboon_245
10 98
predicting:  Baboon_506
10 87
predicting:  Baboon_488
10 60


c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\social_model_class.py:206: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return predictions


In [10]:
groups[i].keys()

dict_keys(['Baboon_199', 'Baboon_153', 'Baboon_245', 'Baboon_200', 'Baboon_101', 'Baboon_395', 'Baboon_349', 'Baboon_285', 'Baboon_324', 'Baboon_289', 'Baboon_502', 'Baboon_296', 'Baboon_410', 'Baboon_390', 'Baboon_518', 'Baboon_206', 'Baboon_506', 'Baboon_74', 'Baboon_388', 'Baboon_298'])

In [ ]:
model.fit()

In [ ]:
model.baboons[0].fit(np.array([1/3]*61))

In [5]:
metadata_df = pd.read_csv(metadata_path)
data_df = pd.read_csv(data_path)

metadata_df["collection_date"] = pd.to_datetime(metadata_df["collection_date"])

In [6]:
metadata_df.set_index('sample', inplace=True)
data_df.set_index('sample', inplace=True)

In [ ]:
model = social_model_class.superModel(data_path,   metadata_path)

In [10]:
metadata_df["collection_date_datetime"] = pd.to_datetime(metadata_df["collection_date"])

In [16]:
metadata_df.drop_duplicates(subset=["baboon_id", "collection_date"], inplace=True)

In [31]:
time_samples_df = metadata_df[["social_group", "collection_date_datetime"]].copy()

In [32]:
time_samples_df["delta_t_cnt"] = 0

In [ ]:
for group in time_samples_df["social_group"].unique():
    for t in time_samples_df["collection_date_datetime"][time_samples_df["social_group"] == group]:
        count = len(time_samples_df[(time_samples_df["social_group"]==group) & (abs(time_samples_df["collection_date_datetime"]-t).dt.days<=10)])
        time_samples_df["delta_t_cnt"][(time_samples_df["social_group"]==group) & (time_samples_df["collection_date_datetime"]==t)] = count
time_samples_df.drop_duplicates(inplace=True)

In [ ]:
for group in time_samples_df["social_group"].unique():
    plt.plot(sorted(time_samples_df["collection_date_datetime"][time_samples_df["social_group"]==group]), time_samples_df["delta_t_cnt"][time_samples_df["social_group"]==group], label=group)


In [ ]:
for t in time_samples_df.index:
    delta_t_datetimes = [i for i in range(t-7,t+7)]
    time_samples_df[""]

In [ ]:
time_samples_df.loc[pd.to_datetime('2000-05-19')]

In [ ]:
resampled_df